The main() function orchestrates the execution of these steps. It loads the data,

* converts H3 indices to Shapely polygons and points
* creates a GeoDataFrame
* reads the OSM network nodes
* converts the CRS of the nodes
* reads the farm data
* finds the closest OSM ID for each point in the GeoDataFrames
* loads the graph
* gets the unique OSM IDs
* calculates the OD matrix
* restructures the OD matrix
* applies cost to the transport cost
* and finally stores the data to pickle files

The script is designed to be run as a **standalone program.**

In [23]:
# Import necessary libraries
from utils.cflp_function import store_data_to_pickle
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import geopandas as gpd
import pandas as pd
import h3
from shapely.ops import nearest_points
from shapely.geometry import Polygon, Point
from fiona.crs import from_epsg

In [24]:
# Define constants
COST_PER_KM = 0.69
MAIN_CRS = 'EPSG:4326'

In [25]:
# Function to load data from a CSV file
def load_data(file_path):
    return pd.read_csv(file_path)

In [26]:
# Function to convert H3 index to Shapely polygon
def cell_to_shapely_polygon(h3_index):
    coords = h3.h3_to_geo_boundary(h3_index)
    flipped = tuple(coord[::-1] for coord in coords)
    return Polygon(flipped)

In [27]:
# Function to convert H3 index to Shapely point
def cell_to_shaply_point(h3_index):
    lat, lon = h3.h3_to_geo(h3_index)
    return Point(lon, lat)

In [28]:
# Function to find the closest OSM ID for each point in a GeoDataFrame
def find_closest_osmid(gdf, n):
    gdf['closest_osmid'] = gdf['geometry'].apply(
        lambda location: n.loc[n['geometry'] == nearest_points(location, n.unary_union)[1], 'osmid'].iloc[0])

In [29]:
# Function to calculate the OD matrix
def calculate_od_matrix(orig, dest, g):
    od_matrix = {}
    for origin in orig:
        od_matrix[origin] = {}
        for destination in dest:
            distance = nx.shortest_path_length(g, origin, destination, weight='length')
            od_matrix[origin][destination] = distance/1000
    return od_matrix


In [30]:
# Function to restructure the OD matrix
def restructure_od(new_nested_dict, loi_gdf):
    placeholders = {i:j for i, j in zip(loi_gdf.index.values, loi_gdf['closest_osmid'])}
    restructured_od = {}

    for farm, distances in new_nested_dict.items():
        restructured_od[farm] = {}
        for index, placeholder in placeholders.items():
            restructured_od[farm][index] = distances.get(placeholder, None)

    transport_cost = {(farm, index): distance for farm, distances in restructured_od.items() for index, distance in distances.items()}
    return transport_cost

    

In [31]:
# Function to apply cost to the transport cost
def apply_cost(transport_cost, cost_per_km=COST_PER_KM):
    return {key: value * cost_per_km for key, value in transport_cost.items()}

In [32]:
# Main function to execute the code
def main():
    # Load data
    loi = load_data('./hex/loi.csv')
    loi['polygon'] = loi['hex9'].apply(lambda x: cell_to_shapely_polygon(x)) 
    loi['geometry'] = loi['hex9'].apply(cell_to_shaply_point)
    loi_gdf = gpd.GeoDataFrame(loi, geometry='geometry', crs=MAIN_CRS)
    n = gpd.read_file(r"./osm_network/G_n.shp")
    n = n.to_crs(MAIN_CRS)
    farm = gpd.read_file("./farm/farm_new.shp")
    
    # Find the closest OSM ID for each point in the GeoDataFrames
    find_closest_osmid(farm, n)
    find_closest_osmid(loi_gdf, n)
    
    # Load the graph
    g = ox.load_graphml('./osm_network/G.graphml') 
    
    # Get the unique OSM IDs
    orig = farm['closest_osmid'].unique().tolist()
    dest = loi_gdf['closest_osmid'].unique().tolist()
    
    # Calculate the OD matrix
    od_matrix = calculate_od_matrix(orig, dest, g)
    
    # Restructure the OD matrix
    new_nested_dict = {idx: od_matrix[row['closest_osmid']] for idx, row in farm.iterrows() if row['closest_osmid'] in od_matrix}
    transport_cost = restructure_od(new_nested_dict, loi_gdf)
    
    # Apply cost to the transport cost
    transport_cost = apply_cost(transport_cost)
    
    # Store the data to pickle
    store_data_to_pickle(transport_cost, 'app_data', 'c_test.pickle')
    store_data_to_pickle(loi_gdf.index.tolist(), 'app_data', 'Plant_test_2.pickle')

In [33]:
# Execute the main function
if __name__ == "__main__":
    main()

hello
